# Open Weather API에서 Current Weather Data API
* https://openweathermap.org/current

## Initialize

In [1]:
import requests
import pprint
from datetime import datetime
import pandas as pd



API_KEY = "bde2a61b68ef64d0ba067f11a64e5d9f"
kelvin =  273.15


### City Name으로 API 호출하기

In [2]:
city_name = "Ansan-si"
res = requests.get(f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={API_KEY}")
pprint.pprint(res.json())

{'base': 'stations',
 'clouds': {'all': 25},
 'cod': 200,
 'coord': {'lat': 37.3236, 'lon': 126.8219},
 'dt': 1632355015,
 'id': 1846918,
 'main': {'feels_like': 292.53,
          'grnd_level': 1015,
          'humidity': 58,
          'pressure': 1016,
          'sea_level': 1016,
          'temp': 292.98,
          'temp_max': 295.12,
          'temp_min': 291.14},
 'name': 'Ansan-si',
 'sys': {'country': 'KR',
         'id': 8105,
         'sunrise': 1632345664,
         'sunset': 1632389360,
         'type': 1},
 'timezone': 32400,
 'visibility': 10000,
 'weather': [{'description': 'scattered clouds',
              'icon': '03d',
              'id': 802,
              'main': 'Clouds'}],
 'wind': {'deg': 310, 'gust': 10.25, 'speed': 6.82}}


### City ID으로 API 호출하기

In [3]:
city_id = "1846918"
res = requests.get(f"https://api.openweathermap.org/data/2.5/weather?id={city_id}&appid={API_KEY}")
data = res.json()
data

{'coord': {'lon': 126.8219, 'lat': 37.3236},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02d'}],
 'base': 'stations',
 'main': {'temp': 293.16,
  'feels_like': 292.73,
  'temp_min': 291.14,
  'temp_max': 295.12,
  'pressure': 1016,
  'humidity': 58,
  'sea_level': 1016,
  'grnd_level': 1015},
 'visibility': 10000,
 'wind': {'speed': 6.54, 'deg': 306, 'gust': 10.17},
 'clouds': {'all': 19},
 'dt': 1632354939,
 'sys': {'type': 1,
  'id': 8105,
  'country': 'KR',
  'sunrise': 1632345664,
  'sunset': 1632389360},
 'timezone': 32400,
 'id': 1846918,
 'name': 'Ansan-si',
 'cod': 200}

### API를 통해서 얻은 Data 추출하기

In [4]:
lat = data["coord"]['lat']
lon = data["coord"]['lon']
temperature_feels = round(data["main"]['feels_like'] - kelvin,2)
temperature = round(data["main"]['temp'] - kelvin,2)
temperature_max = round(data["main"]['temp_max'] - kelvin,2)
temperature_min = round(data["main"]['temp_min'] - kelvin,2)
pressure = data["main"]["pressure"]
humidity = data["main"]['humidity']

#### 현재 시간 가져오기

In [5]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
current_time


'08:59:55'

In [6]:
data = {
    "current_time":current_time,
    "temperature":temperature,
    "temperature_feels":temperature_feels,
    "temperature_max":temperature_max,
    "temperature_min":temperature_min,
    "pressure":pressure,
}

In [7]:
df = pd.DataFrame(data,index = [1])
df.columns.name = city_name
df

Ansan-si,current_time,temperature,temperature_feels,temperature_max,temperature_min,pressure
1,08:59:55,20.01,19.58,21.97,17.99,1016


### 대기 오염 가져오기

In [8]:

res = requests.get(f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API_KEY}")
data = res.json()
pprint.pprint(data)

{'coord': {'lat': 37.3236, 'lon': 126.8219},
 'list': [{'components': {'co': 494,
                          'nh3': 2.41,
                          'no': 5.42,
                          'no2': 22.96,
                          'o3': 49.35,
                          'pm10': 23.16,
                          'pm2_5': 17.71,
                          'so2': 24.32},
           'dt': 1632355200,
           'main': {'aqi': 2}}]}


In [9]:
w_data = data['list'][0]['components']

#일산화 탄소
CO = w_data['co']

#일산화 질소
NO = w_data['no']

#오존
O3 = w_data['o3']

#미세먼지
pm = w_data['pm2_5']

#초미세먼지
pm10 = w_data['pm10']

di = {
    "일산화 탄소":CO,
    "일산화 질소":NO,
    "오존":O3,
    "미세먼지":pm,
    "초미세먼지":pm10,

}

df = pd.DataFrame(data = di,index = [city_name])
df.columns.name = "대기오염"
df

대기오염,일산화 탄소,일산화 질소,오존,미세먼지,초미세먼지
Ansan-si,494,5.42,49.35,17.71,23.16
